In [138]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessor, LogitsProcessorList
import torch
import numpy as np
import os
import re

model_name = "eryk-mazus/polka-1.1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [48]:
vocab = tokenizer.get_vocab()
print(f"Vocab size: {len(vocab)}")
print(tokenizer.all_special_ids)

print([tokenizer.decode([i]) for i in tokenizer.all_special_ids])

Vocab size: 43882
[1, 2, 0]
['<s>', '</s>', '<unk>']


In [162]:
class SameLetterLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer, target_letter):
        self.tokenizer = tokenizer
        self.target_letter = target_letter.lower()
        self.separators = {".", ",", "!", "?", " "}

        self.forbidden_token_ids = []
        self.sus_token_ids = []

        vocab = tokenizer.get_vocab()
        for token, idx in vocab.items():
            if idx in tokenizer.all_special_ids:
                continue

            decoded = tokenizer.decode([idx])
            if not decoded:
                continue

            starts_with_space = "\u2581" in token or decoded.startswith(" ")
            starts_with_punct = (not decoded[0].isalpha())

            letters_only = "".join(filter(str.isalpha, decoded))
            counts = [decoded.count(c) for c in self.separators]
            is_clean = all(c.isalpha() or c.isspace() or c in self.separators for c in decoded)

            repeating_sep = any(cnt > 1 for cnt in counts)
            has_sep = any(cnt > 0 for cnt in counts)
            if (not starts_with_space and decoded[0] not in self.separators and decoded[0] and has_sep) or repeating_sep:
                is_clean = False

            if not is_clean:
                self.forbidden_token_ids.append(idx)
                continue

            if not letters_only:
                continue

            first_char = letters_only[0].lower()
            if first_char == self.target_letter:
                continue



            if starts_with_punct  or starts_with_space:
                self.forbidden_token_ids.append(idx)
                continue

            self.sus_token_ids.append(idx)

    def __call__(self, input_ids, scores):
        scores[:, self.forbidden_token_ids] = -float("inf")
        batch_size = input_ids.shape[0]

        for i in range(batch_size):
            last_token_id = input_ids[i, -1].item()
            last_token_decoded = self.tokenizer.decode([last_token_id])
            if not last_token_decoded:
                is_boundary = True
            else:
                is_boundary = last_token_decoded[-1] in self.separators or last_token_decoded[-1].isspace()
            if is_boundary:
                scores[i, self.sus_token_ids] = -float("inf")

        return scores

In [149]:
def generate(prefix):
    words = prefix.split()
    target_letter = words[-1][0].lower()

    processor = SameLetterLogitsProcessor(tokenizer, target_letter)

    inputs = tokenizer(prefix, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        top_k=100,
        top_p=0.90,
        repetition_penalty=1.2,
        num_return_sequences=10,
        logits_processor=[processor],
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded_candidates = tokenizer.batch_decode(outputs, skip_special_tokens=True)


    return decoded_candidates

In [166]:
prefixes = [
    "W wyniku wskazanych",
]

for p in prefixes:
    candidates = generate(p)
    for i, c in enumerate(candidates):
        print(f"Candidate {i}: {c}")

Candidate 0: W wyniku wskazanych wyżej wpadek w w wejściu, wieży, wieżowiec, włącz
Candidate 1: W wyniku wskazanych wyżej wskaźników wzrostu, wynagrodzenie wskoczyło w tymże,
Candidate 2: W wyniku wskazanych wniosków, wojewoda wielkopolski wydał w dniu wydania w dniach ww. 
Candidate 3: W wyniku wskazanych wskaźników wzrostowych, webshop . Ważккѕkсyкк
Candidate 4: W wyniku wskazanych wyżej wad, wpływających wprost wpłynęło w dalszym 
Candidate 5: W wyniku wskazanych wymogów,  w wielu większych wymianach wielu
Candidate 6: W wyniku wskazanych wyżej względów, wszelkie wypłaty wygranej w wersji wirtual
Candidate 7: W wyniku wskazanych wyżej względów, wszelkie wymienione we wniosku warunki warunek 
Candidate 8: W wyniku wskazanych wyżej wytycznych wartość wywołania wartościowej wersji witryny
Candidate 9: W wyniku wskazanych wniosków wydano w dalszym . Ważńciu Wojewody Warmińsko
